In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.models import Model

import xgboost
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.metrics import roc_curve, auc,  precision_score, recall_score, accuracy_score, f1_score
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import seaborn as sns
import pickle


os.environ['KMP_DUPLICATE_LIB_OK']='True'
import PIL
from PIL import ImageFile, Image
ImageFile.LOAD_TRUNCATED_IMAGES = True
import pathlib
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
from itertools import chain

In [174]:
no_time = pd.read_csv('../data/notime_df.csv')
no_time = no_time.loc[:, ~no_time.columns.str.contains('^Unnamed')]
no_time = no_time.dropna()

In [179]:
class Fire(object):
    def __init__(self, df):
        self.y = df.pop('Target')
        self.X = df

    def split(self):
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, stratify=self.y)

    def fit(self, model):
        self.model = model
        kf = KFold(n_splits=5)
        
        accuracy = []
        precision = []
        recall = []
        f1 = []

        for train_index, test_index in kf.split(self.X_train):
            X_train_split, X_test_split = self.X_train.iloc[train_index], self.X_train.iloc[test_index]
            y_train_split, y_test_split = self.y_train.iloc[train_index], self.y_train.iloc[test_index]
            self.model.fit(X_train_split, y_train_split, verbose=True)
            self.pred = self.model.predict(X_test_split)

            assess = lambda method, val=y_test_split, pred=self.pred: method(val, pred)

            accuracy.append(assess(accuracy_score))
            precision.append(assess(precision_score))
            recall.append(assess(recall_score))
            f1.append(assess(f1_score))
        
        return model
        

In [180]:
params = {'max_depth': 10,
          'colsample_bytree': 0.4,
          'subsample': 0.8,
          'min_child_weight': 6,
          'n_estimators': 1000, 
          'learning_rate': 0.01,
          'lambda': 0.8,
          'alpha': 0.5}
boost = XGBClassifier()
boost = XGBClassifier(**params)

In [181]:
data_boost = Fire(no_time)
data_boost.split()
fitted_boost = data_boost.fit(boost)

In [188]:
pickle.dump(fitted_boost, open("pima.pickle.dat", "wb"))

In [184]:
norcal = pd.read_csv('../data/norcal.csv')
norcal = norcal.loc[:, ~norcal.columns.str.contains('^Unnamed')]
socal = pd.read_csv('../data/socal.csv')
socal = socal.loc[:, ~socal.columns.str.contains('^Unnamed')]

# Input a date and select a region

In [2]:
def combinedModels():
    region = input('Enter "norcal" or "socal":  ')
    date = input('Enter a date between 1/1/2018 and 9/13/2020:  ')
    boost = pickle.load(open("pima.pickle.dat", "rb"))
    xception = load_model('xception_50epoch.h5')
    batch_size = 16
    img_height = 256
    img_width = 256
    epochs=50
    
    if region == 'norcal':
        norcal = pd.read_csv('../data/norcal.csv')
        norcal = norcal.loc[:, ~norcal.columns.str.contains('^Unnamed')]
        df = norcal[norcal['Date']==date]
        df.drop('Date', axis=1, inplace=True)

        # reformat date: 9/1/2020 -> 2020-09-01
        datetime = pd.to_datetime(date, format='%m/%d/%Y')
        datetime = datetime.strftime('%Y-%m-%d')
        # find it in cleaned_data image files
        path = glob.glob(f'../data/cleaned_data/*/{datetime}_1.jpg')
        path = ''.join(element for element in path)
        image = tf.keras.preprocessing.image.load_img(path, interpolation="bilinear")
        input_arr = keras.preprocessing.image.img_to_array(image)
        input_arr = np.array([input_arr])  # Convert single image to a batch.
        image_risk = xception.predict(input_arr,batch_size=batch_size)
        
    elif region == 'socal':
        socal = pd.read_csv('../data/socal.csv')
        socal = socal.loc[:, ~socal.columns.str.contains('^Unnamed')]
        df = socal[socal['Date']==date]
        df.drop('Date', axis=1, inplace=True)
        
        # reformat date: 9/1/2020 -> 2020-09-01
        datetime = pd.to_datetime(date, format='%m/%d/%Y')
        datetime = datetime.strftime('%Y-%m-%d')
        # find it in cleaned_data image files
        path = glob.glob(f'../data/cleaned_data/*/{datetime}.jpg')
        path = ''.join(element for element in path)
        image = tf.keras.preprocessing.image.load_img(path, interpolation="bilinear")
        input_arr = keras.preprocessing.image.img_to_array(image)
        input_arr = np.array([input_arr])  # Convert single image to a batch.
        image_risk = xception.predict(input_arr,batch_size=batch_size)
        
    conditions_y = df.pop('Target')
    conditions_y_pred = boost.predict(df)
    conditions_risk = np.mean(conditions_y_pred)
    
    risk = .8*(image_risk.ravel()[0]) + 0.2*conditions_risk
    print(image_risk, conditions_risk)
    return print(f'Risk for fire in {region} on {date} was: {round(risk,2)}')

In [ ]:
combinedModels()

Enter "norcal" or "socal":  norcal


In [16]:
date = '9/3/2020'
datetime = pd.to_datetime(date, format='%m/%d/%Y')
datetime = datetime.strftime('%Y-%m-%d')

In [17]:
datetime

'2020-09-03'

In [27]:
path = glob.glob(f'../data/cleaned_data/*/{datetime}_1.jpg')
path = ''.join(element for element in path)
image = tf.keras.preprocessing.image.load_img(path, interpolation="bilinear")
input_arr = keras.preprocessing.image.img_to_array(image)

In [28]:
input_arr

array([[[185., 187., 186.],
        [185., 187., 186.],
        [186., 188., 187.],
        ...,
        [ 70.,  62.,  43.],
        [ 70.,  62.,  43.],
        [ 70.,  62.,  43.]],

       [[187., 189., 188.],
        [188., 190., 189.],
        [188., 190., 189.],
        ...,
        [ 66.,  58.,  39.],
        [ 66.,  58.,  39.],
        [ 66.,  58.,  39.]],

       [[190., 192., 191.],
        [190., 192., 191.],
        [189., 191., 190.],
        ...,
        [ 67.,  59.,  40.],
        [ 67.,  59.,  40.],
        [ 67.,  59.,  40.]],

       ...,

       [[119., 122., 131.],
        [117., 120., 129.],
        [117., 120., 129.],
        ...,
        [119., 111.,  98.],
        [119., 111.,  98.],
        [120., 112.,  99.]],

       [[112., 115., 124.],
        [111., 114., 123.],
        [111., 114., 123.],
        ...,
        [120., 112.,  99.],
        [121., 113., 100.],
        [122., 114., 101.]],

       [[114., 117., 126.],
        [112., 115., 124.],
        [110., 1

In [ ]:
# Define inputs: image and conditions
# include date prior to both these inputs to specify
# norcal or socal and where in conditions df to look

date_input = keras.Input(input('Enter a date between 1/1/2018 and 9/13/2020:  '))
region_input = keras.Input(input('Enter "norcal" or "socal":  '))

image_input = keras.Input(
    shape=(img_height, img_width, 3), 
    name="satellite_image"
    ) 

conditions_input = keras.Input(
    shape=(None,), 
    name="conditions")


xception_model=load_model('xception_50epoch.h5')(image_input)
xception = layers.xception_model()(image_input)

probability1 = xception
probability2 = 

fire_risk = layers.average(x)

bigmodel = keras.Model(
    inputs = [image_input, conditions_input, date_input]
    outputs = fire_risk)